# Librairies

In [1]:
import project_functions as pf
import pandas as pd
import yfinance as yf

# Portfolio Analysis


Importons les données :

In [2]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
prices = pd.read_csv('datas/prices.csv', sep=',')
prices = prices.set_index('Date')

## Analyse des returns

Nous ne prenons uniquement les assets avec des returns anualisés positifs.

In [3]:
# Calculons les rendements journaliers
daily_returns = pf.daily_asset_return(prices)

# Calculons les rendements annuels
annualy_returns = pf.annualy_asset_return(daily_returns)

# On récupère les tickers des index avec rendements positifs
tickers_pos = annualy_returns.index.to_list()

Actualisons le dataframe prices avec uniquement les tickers avec un rendement positif

In [4]:
begin_train = "2014-01-01"
end_train = "2017-01-01"
begin_test = end_train
end_test = prices.index[-1]
prices_train, prices_test = pf.seperate_df(prices, begin_train, end_train, begin_test, end_test)

## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

In [5]:
# récupérons les données liées aux critères ESG sauvegardés dans le fichier esg_scores.csv
esg = pd.read_csv('datas/esg_scores.csv', sep=',')

# supprimons les lignes tickers avec un return négatif
esg = esg[esg['Ticker Yahoo'].isin(tickers_pos)]

# On ne conserve que les tickers avec des scores ESG différents de NaN
esg = esg.dropna()

n = len(esg)

print(f'Nombre de tickers avec des scores ESG : {n}')

Nombre de tickers avec des scores ESG : 39


In [6]:
# On conserve les tickers avec un score environnemental supérieur à la médiane
esg = esg.sort_values(by=['Environment Score'])
tickers_env = esg.iloc[int(n*1/4):]
tickers_env

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
60,60,VASTN.AS,52.0,45.0,74.0,57.0
65,65,WKL.AS,54.0,77.0,69.0,69.0
24,24,FUR.AS,57.0,90.0,84.0,77.0
15,15,BESI.AS,61.0,69.0,78.0,69.0
10,10,ASM.AS,62.0,78.0,83.0,74.0
48,48,RAND.AS,63.0,81.0,76.0,75.0
7,7,AMG.AS,65.0,71.0,92.0,74.0
11,11,ASML.AS,66.0,85.0,88.0,80.0
2,2,AD.AS,68.0,81.0,88.0,80.0
0,0,AALB.AS,71.0,75.0,58.0,69.0


In [7]:
# On conserve les tickers avec un score social supérieur à la médiane
esg = esg.sort_values(by=['Social Score'])
tickers_soc = esg.iloc[int(n*1/4):]
tickers_soc

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
64,64,WHA.AS,85.0,67.0,51.0,69.0
34,34,INGA.AS,86.0,68.0,89.0,78.0
57,57,TWEKA.AS,49.0,69.0,67.0,61.0
15,15,BESI.AS,61.0,69.0,78.0,69.0
35,35,KENDR.AS,39.0,70.0,60.0,57.0
7,7,AMG.AS,65.0,71.0,92.0,74.0
19,19,CRBN.AS,78.0,75.0,68.0,74.0
0,0,AALB.AS,71.0,75.0,58.0,69.0
36,36,KPN.AS,77.0,76.0,67.0,74.0
65,65,WKL.AS,54.0,77.0,69.0,69.0


In [8]:
# Concaténation des dataframes en supprimant les doublons
tickers_esg = pd.merge(tickers_env, tickers_soc, how ='inner', on =['Ticker Yahoo'])
tickers_esg

,A_x,Ticker Yahoo,Environment Score_x,Social Score_x,Governance Score_x,Total Score_x,A_y,Environment Score_y,Social Score_y,Governance Score_y,Total Score_y
0,65,WKL.AS,54.0,77.0,69.0,69.0,65,54.0,77.0,69.0,69.0
1,24,FUR.AS,57.0,90.0,84.0,77.0,24,57.0,90.0,84.0,77.0
2,15,BESI.AS,61.0,69.0,78.0,69.0,15,61.0,69.0,78.0,69.0
3,10,ASM.AS,62.0,78.0,83.0,74.0,10,62.0,78.0,83.0,74.0
4,48,RAND.AS,63.0,81.0,76.0,75.0,48,63.0,81.0,76.0,75.0
5,7,AMG.AS,65.0,71.0,92.0,74.0,7,65.0,71.0,92.0,74.0
6,11,ASML.AS,66.0,85.0,88.0,80.0,11,66.0,85.0,88.0,80.0
7,2,AD.AS,68.0,81.0,88.0,80.0,2,68.0,81.0,88.0,80.0
8,0,AALB.AS,71.0,75.0,58.0,69.0,0,71.0,75.0,58.0,69.0
9,5,AKZA.AS,72.0,80.0,87.0,78.0,5,72.0,80.0,87.0,78.0


In [9]:
# Récupérons la liste des tickers
list_esg = tickers_esg['Ticker Yahoo'].to_list()
print(f'Nombre de tickers ESG sélectionnés : {len(list_esg)}')

Nombre de tickers ESG sélectionnés : 27


## Étude du ROA

On ne sélectionne uniquement les tickers avec le meilleur ROA

In [10]:
# on récupère les ROA des tickers sélectionnés précédemment et on les stocke dans un csv
# Lance cette fonction que si on souhaite les dernières valeurs des ROA
# pf.get_ROA(list_esg)

ROA = pd.read_csv('datas/ROA.csv', sep=',')
ROA.set_index('Unnamed: 0', inplace=True)

In [11]:
# On ne conserve que les tickers avec un ROA supérieur à la médiane
tickers_ROA = ROA.iloc[:int(len(ROA)*1/2)]
tickers_ROA

,ROA
Unnamed: 0,
BESI.AS,0.17315
ASML.AS,0.12882
ASM.AS,0.12209
MT.AS,0.10371
REN.AS,0.09145
AMG.AS,0.08967
WKL.AS,0.08048
SHELL.AS,0.07826
RAND.AS,0.06655


In [12]:
# On récupère la liste des tickers ESG sélectionnés
tickers_selected = tickers_ROA.index.to_list()
print(f'Nombre de tickers ESG sélectionnés : {len(tickers_selected)}')

Nombre de tickers ESG sélectionnés : 13


## Enregistrement des tickers

Nous devons enregistrer les tickers sélectionnés dans le fichier tickers.txt

In [13]:
pf.tickers_storage(tickers_selected, 'datas/tickers.txt')